In [ ]:
from google.colab import files
train_movie = files.upload()

Saving movies_train.csv to movies_train.csv


In [ ]:
import pandas as pd
df = pd.read_csv('/content/movies_train.csv')
# type(df)

In [ ]:
from numpy.core.fromnumeric import mean
#평균 계산
box_mean = mean(df.box_off_num)

# 개선후_df_high와 df_low
df_over_mean = df[(df["box_off_num"]>=box_mean)]
df_lower_mean = df[(df["box_off_num"]<=box_mean)]
df_high = df_over_mean.iloc[:df_over_mean.shape[0],9:12] #높은 관객수 (샘플 30, 내림차순)
df_low = df_lower_mean.iloc[:df_lower_mean.shape[0], 9:12] # 낮은 관객수 (샘플 30, 오름차순)

# 개선전_df_high와 df_low
# data_size = df.shape[0]
# df_high = (df.sort_values(by = "box_off_num", ascending = False)).iloc[:data_size,9:12] #높은 관객수 (샘플 30, 내림차순)
# df_low = (df.sort_values(by = "box_off_num", ascending = True)).iloc[:data_size,9:12] # 낮은 관객수 (샘플 30, 오름차순)
# df_high.shape
# df_low

In [ ]:
#staff, actor list 만들기
import numpy as np
high_staff = df_high.num_staff[:]
high_actor = df_high.num_actor[:]
low_staff = df_low.num_staff[:]
low_actor = df_low.num_actor[:]
# actor_num = df.num_actor[1:50]
high_staff_list = high_staff.tolist()
high_actor_list = high_actor.tolist()
low_staff_list = low_staff.tolist()
low_actor_list = low_actor.tolist()
# type(staff_num_list)
# type(actor_num_list)
# high_staff_list

In [ ]:
#/2차원 배열로 만들기
high_data = np.column_stack((high_staff_list,high_actor_list))
# high_data
low_data = np.column_stack((low_staff_list,low_actor_list))
# low_data

# movie_input - [[staff_num, actor_num],] : 높은 관객 수 50 샘플, 낮은 관객 수 50 샘플 -> input data
movie_input = np.concatenate((high_data, low_data))

In [ ]:
#타깃 생성 (50) : 높은 관객 수가 예측되면 1, 낮은 관객수가 예측되면 0
# ex) fish_target = np.concatenate((np.ones(35), np.zeros(14)))
movie_target =  np.concatenate((np.ones(df_over_mean.shape[0]), np.zeros(df_lower_mean.shape[0]))) #-> target data

# movie_target.shape

In [ ]:
#input data와 target data 합치기
movie_data = np.column_stack((movie_input, movie_target))
# movie_data

In [ ]:
from sklearn.model_selection import train_test_split
#훈련세트 테스트세트 나누기
train_input, test_input, train_target, test_target = train_test_split(
    movie_input, movie_target, stratify=movie_target, random_state=df_over_mean.shape[0])
# train_input
# test_input

In [ ]:
#scale 처리 <- actor에 배해 staff의 범위가 크므로, 스케일 처리 필요
mean = np.mean(train_input, axis=0)
std = np.std(train_input, axis=0)

train_scaled = (train_input - mean) / std
test_scaled = (test_input - mean) / std

1. k-최근접 이웃 분류 : 스탭 수와 배우 수를 주면 관객수가 높은지 낮은지 분류**
2. k-최근접 이웃 회귀 : 스탭 수와 배우 수를 주면 관객수를 예측하는 모델

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier()
kn.fit(train_scaled, train_target)
train_score = kn.score(train_scaled, train_target)
test_score = kn.score(test_scaled, test_target)
# new = [200, 0]
# print(kn.predict([new]))

In [ ]:
train_score * 100

88.88888888888889

In [ ]:
test_score * 100

83.33333333333334